<a href="https://colab.research.google.com/github/Madhura0802/Vector-Addition/blob/main/Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [11]:
code = """
#include <stdio.h>
#include <stdlib.h>
#define N 1000000
__global__ void add(int *a, int *b, int *c) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < N) {
c[tid] = a[tid] + b[tid];
}
}
int main() {
int *a, *b, *c;
int *d_a, *d_b, *d_c;
int size = N * sizeof(int);
// Allocate memory on host
a = (int*)malloc(size);
b = (int*)malloc(size);
c = (int*)malloc(size);
// Initialize arrays
for (int i = 0; i < N; i++) {
a[i] = i;
b[i] = i * 2;
}
// Allocate memory on device
cudaMalloc(&d_a, size);
cudaMalloc(&d_b, size);
cudaMalloc(&d_c, size);
// Copy data from host to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
// Launch kernel with 1 million threads
add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);
// Copy result from device to host
cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
// Print first and last elements of result
printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);
// Free memory
free(a);
free(b);
free(c);
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}
"""


In [12]:
text_file = open("Vector.cu","w")
text_file.write(code)
text_file.close()


In [13]:
!nvcc Vector.cu

In [14]:
!./a.out

c[0]=0, c[999999] = 2999997

In [15]:
!nvprof ./a.out

==2086== NVPROF is profiling process 2086, command: ./a.out
==2086== c[0]=0, c[999999] = 2999997Profiling application: ./a.out
==2086== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.57%  1.5775ms         1  1.5775ms  1.5775ms  1.5775ms  [CUDA memcpy DtoH]
                   47.92%  1.4950ms         2  747.49us  703.47us  791.50us  [CUDA memcpy HtoD]
                    1.51%  47.135us         1  47.135us  47.135us  47.135us  add(int*, int*, int*)
      API calls:   97.01%  223.47ms         3  74.488ms  92.123us  223.27ms  cudaMalloc
                    2.08%  4.7886ms         3  1.5962ms  950.19us  2.8591ms  cudaMemcpy
                    0.59%  1.3556ms         1  1.3556ms  1.3556ms  1.3556ms  cuDeviceGetPCIBusId
                    0.23%  535.88us         3  178.63us  134.49us  201.36us  cudaFree
                    0.06%  135.69us       101  1.3430us     128ns  52.948us  cuDeviceGetAttribute
                